#### Importing Required packages

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk

In [3]:
df = pd.read_csv(r"E:\Data Science\Project\CommonLit Readability Prize\train.csv")
df.head()

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845


In [4]:
df1 = df 

In [5]:
df.target.describe()

count    2834.000000
mean       -0.959319
std         1.033579
min        -3.676268
25%        -1.690320
50%        -0.912190
75%        -0.202540
max         1.711390
Name: target, dtype: float64

#### only taking imp columns

In [6]:
df = df.iloc[:, [-3,-2]]

In [7]:
df.head()

,excerpt,target
0,When the young people returned to the ballroom...,-0.340259
1,"All through dinner time, Mrs. Fayre was somewh...",-0.315372
2,"As Roger had predicted, the snow departed as q...",-0.580118
3,And outside before the palace a great garden w...,-1.054013
4,Once upon a time there were Three Bears who li...,0.247197


In [8]:
df.isnull().sum()

excerpt    0
target     0
dtype: int64

In [9]:
df.shape

(2834, 2)

In [10]:
df = df.rename(columns={'excerpt':'data'})

In [11]:
df.head()

,data,target
0,When the young people returned to the ballroom...,-0.340259
1,"All through dinner time, Mrs. Fayre was somewh...",-0.315372
2,"As Roger had predicted, the snow departed as q...",-0.580118
3,And outside before the palace a great garden w...,-1.054013
4,Once upon a time there were Three Bears who li...,0.247197


#### Removing stopwords and punctuation marks

In [12]:
import string

In [13]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [14]:
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

import spacy
import en_core_web_sm 
lang = spacy.load("en_core_web_sm")  

In [15]:
l1 = list(stopwords.words('english'))
l2 = list(ENGLISH_STOP_WORDS)
l3 = list(lang.Defaults.stop_words)

In [16]:
l1.extend(l2)

In [17]:
l1.extend(l3)

In [18]:
len(l1)

823

In [19]:
stop_words = set(l1)

In [21]:
def text_process(mess):            ### creating a function
    """                                                        ## a docstring
    1. remove the punctuation
    2. remove the stopwords
    3. return the list of clean textwords
    
    """
    nopunc = [char for char in mess if char  not in string.punctuation]
    nopunc = "".join(nopunc)
    
    return [ word for word in nopunc.split() if word not in stop_words]

#### Word Embedding

In [22]:
from sklearn.feature_extraction.text import CountVectorizer  

In [23]:
bow_transformer = CountVectorizer(analyzer= text_process).fit(df['data'])

In [24]:
tdm = bow_transformer.transform(df['data'])  ## creating TDM

In [25]:
tdm.shape

(2834, 32904)

#### Data  sampling

In [26]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(tdm, df.target, test_size=.2)

In [27]:
train_x.shape

(2267, 32904)

In [28]:
train_y.shape

(2267,)

In [29]:
train_y.head()

17     -0.617733
894    -0.898353
1953   -0.121105
1045    0.332933
502     0.533262
Name: target, dtype: float64

#### Linear Regression 

In [35]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [36]:
lr.fit(train_x, train_y)

LinearRegression()

In [37]:
lr.score(train_x, train_y)

0.9998935619525472

In [38]:
Rsqu =lr.score(train_x  , train_y)
N = train_x.shape[0] # NUMBER OF  records
K = train_x.shape[1] # NUMBER OF X variable 
AdjRsqure  = 1  - (1-Rsqu)  * ( N-1) / (N-K-1)
print("Value of AdjRsqure is"  , AdjRsqure)


Value of AdjRsqure is 1.0000078722049588


In [39]:
pred = lr.predict(test_x)

In [40]:
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error

In [42]:
rmse_lr = np.sqrt(mean_squared_error(test_y, pred))
rmse_lr

0.7968608105996889

#### Decision Tree Regressor

In [49]:
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor()

In [50]:
dt.fit(train_x, train_y)

DecisionTreeRegressor()

In [51]:
dt.score(train_x, train_y)

0.9998935619525472

In [52]:
pred_dt = dt.predict(test_x)

In [55]:
rmse_dt = np.sqrt(mean_squared_error(test_y, pred_dt))
rmse_dt

1.105567618666046

#### Random forest regressor

In [57]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()

In [58]:
rfr.fit(train_x, train_y)

RandomForestRegressor()

In [59]:
rfr.score(train_x, train_y)

0.9067003430140727

In [60]:
pred_rfr = rfr.predict(test_x)

In [63]:
rmse_rfr = np.sqrt(mean_squared_error(test_y, pred_rfr))
rmse_rfr

0.8753753629489911

#### Cat boost regressor

In [66]:
from catboost import CatBoostRegressor

In [67]:
cbc = CatBoostRegressor(max_depth=10)

In [68]:
cbc.fit(train_x, train_y)

Learning rate set to 0.046595
0:	learn: 1.0129429	total: 1.12s	remaining: 18m 35s
1:	learn: 1.0067684	total: 2.1s	remaining: 17m 29s
2:	learn: 1.0021871	total: 3.01s	remaining: 16m 40s
3:	learn: 0.9974316	total: 3.75s	remaining: 15m 33s
4:	learn: 0.9942580	total: 4.7s	remaining: 15m 35s
5:	learn: 0.9896631	total: 5.88s	remaining: 16m 14s
6:	learn: 0.9849712	total: 6.61s	remaining: 15m 38s
7:	learn: 0.9806875	total: 7.33s	remaining: 15m 9s
8:	learn: 0.9774552	total: 8.08s	remaining: 14m 49s
9:	learn: 0.9733791	total: 8.83s	remaining: 14m 34s
10:	learn: 0.9691445	total: 9.58s	remaining: 14m 21s
11:	learn: 0.9647127	total: 10.3s	remaining: 14m 11s
12:	learn: 0.9607148	total: 11.1s	remaining: 14m 1s
13:	learn: 0.9570465	total: 11.8s	remaining: 13m 51s
14:	learn: 0.9539382	total: 12.6s	remaining: 13m 44s
15:	learn: 0.9507734	total: 13.3s	remaining: 13m 39s
16:	learn: 0.9474238	total: 14.1s	remaining: 13m 34s
17:	learn: 0.9447552	total: 14.9s	remaining: 13m 32s
18:	learn: 0.9410732	total: 15

In [69]:
cbc.score(train_x, train_y)

0.9061739624987026

In [70]:
pred_cbc = cbc.predict(test_x)

In [71]:
mean_absolute_percentage_error(test_y, pred_cbc)

5525704603826.897

In [72]:
rmse_cbc = np.sqrt(mean_squared_error(test_y, pred_cbc))
rmse_cbc

0.7930786013232363

#### Gradient boost regressor

In [73]:
from sklearn.ensemble import GradientBoostingRegressor

gbc = GradientBoostingRegressor(max_depth=16)

In [74]:
gbc.fit(train_x, train_y)

GradientBoostingRegressor(max_depth=16)

In [75]:
gbc.score(train_x, train_y)

0.9749986008709297

In [76]:
pred_gbc = gbc.predict(test_x)

In [77]:
rmse_gbc = np.sqrt(mean_squared_error(test_y, pred_gbc))
rmse_gbc

0.8915470251731155

In [78]:
from xgboost import XGBRFRegressor
xgb = XGBRFRegressor()

In [79]:
xgb.fit(train_x, train_y)

XGBRFRegressor(base_score=0.5, booster='gbtree', callbacks=None,
               colsample_bylevel=1, colsample_bytree=1,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
               importance_type=None, interaction_constraints='', max_bin=256,
               max_cat_to_onehot=4, max_delta_step=0, max_depth=6, max_leaves=0,
               min_child_weight=1, missing=nan, monotone_constraints='()',
               n_estimators=100, n_jobs=0, num_parallel_tree=100,
               objective='reg:squarederror', predictor='auto', random_state=0,
               reg_alpha=0, sampling_method='uniform', scale_pos_weight=1, ...)

In [80]:
xgb.score(train_x, train_y)

0.3357857625680507

In [81]:
pred_xgb = xgb.predict(test_x)

In [83]:
rmse_xgb = np.sqrt(mean_squared_error(test_y, pred_xgb))
rmse_xgb

0.9764332744848206

In [84]:
## 6.To remove outliers         

In [85]:
def remove_outliers(df, col, k):
    mean = df[col].mean()
    global df1
    sd= df[col].std()
    final_list =[x for x in df[col] if (x > mean-k*sd)]
    final_list =[x for x in final_list if (x < mean+k*sd)]
    # print (final_list); df=df.loc[df[col].isin(final_list)]; print(df.shape); return df
    df1= df.loc[df[col].isin(final_list)]; print(df1.shape)
    print("Number of Outliers removed==>" , df.shape[0]- df1.shape[0])


In [86]:
df2 = df

In [87]:
df2.head()

,data,target
0,When the young people returned to the ballroom...,-0.340259
1,"All through dinner time, Mrs. Fayre was somewh...",-0.315372
2,"As Roger had predicted, the snow departed as q...",-0.580118
3,And outside before the palace a great garden w...,-1.054013
4,Once upon a time there were Three Bears who li...,0.247197


In [88]:
remove_outliers(df2, "target", 2.5)

(2824, 2)
Number of Outliers removed==> 10


In [203]:
df1.shape

(2824, 2)

#### Using RNN & LSTM

In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from keras.layers import Embedding, LSTM
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense,Dropout

In [96]:
df.head()

,data,target
0,When the young people returned to the ballroom...,-0.340259
1,"All through dinner time, Mrs. Fayre was somewh...",-0.315372
2,"As Roger had predicted, the snow departed as q...",-0.580118
3,And outside before the palace a great garden w...,-1.054013
4,Once upon a time there were Three Bears who li...,0.247197


In [97]:
## sampling

In [206]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df1, test_size=0.2)

In [207]:
train_x3 = train.iloc[:, 0]
train_y3 = train.iloc[:, 1]

test_x3 = test.iloc[:, 0]
test_y3 = test.iloc[:, 1]

In [208]:
max_num_words = 10000 
seq_len = 50 
embedding_size = 100

In [209]:
from keras.preprocessing.text import Tokenizer  
from keras_preprocessing.sequence import pad_sequences

In [210]:
tokenizer = Tokenizer(num_words = max_num_words)
tokenizer.fit_on_texts(df.data)
train_x3   = tokenizer.texts_to_sequences(train_x3)
train_x3   = pad_sequences(train_x3  , maxlen = seq_len)

In [211]:
test_x3 = tokenizer.texts_to_sequences(test_x3)
test_x3 = pad_sequences(test_x3, maxlen = seq_len)

In [212]:
## now lets create an architecture

In [213]:
model = Sequential()
model.add(Embedding(input_dim = max_num_words, 
                    input_length = seq_len,
                     output_dim = embedding_size))
model.add(Dense(128 , activation = 'relu'))

model.add(LSTM(1))
model.add(Dropout(0.25))
model.add(Dense(1 , activation= 'linear')) 
model.compile(optimizer='adam', loss = 'mean_squared_error' , metrics = ['mse'])

In [214]:
model.fit(train_x3, train_y3, epochs=10, batch_size=50, verbose=2, shuffle=False)

Epoch 1/10
46/46 - 6s - loss: 1.7966 - mse: 1.7966 - 6s/epoch - 141ms/step
Epoch 2/10
46/46 - 1s - loss: 1.4944 - mse: 1.4944 - 1s/epoch - 29ms/step
Epoch 3/10
46/46 - 1s - loss: 1.3021 - mse: 1.3021 - 1s/epoch - 30ms/step
Epoch 4/10
46/46 - 1s - loss: 1.2077 - mse: 1.2077 - 1s/epoch - 30ms/step
Epoch 5/10
46/46 - 1s - loss: 1.1257 - mse: 1.1257 - 1s/epoch - 30ms/step
Epoch 6/10
46/46 - 1s - loss: 1.0144 - mse: 1.0144 - 1s/epoch - 31ms/step
Epoch 7/10
46/46 - 1s - loss: 0.9171 - mse: 0.9171 - 1s/epoch - 31ms/step
Epoch 8/10
46/46 - 1s - loss: 0.8467 - mse: 0.8467 - 1s/epoch - 30ms/step
Epoch 9/10
46/46 - 1s - loss: 0.7514 - mse: 0.7514 - 1s/epoch - 29ms/step
Epoch 10/10
46/46 - 1s - loss: 0.7036 - mse: 0.7036 - 1s/epoch - 30ms/step


In [215]:
pred_lstm = model.predict(test_x3)

18/18 [==============================] - 1s 8ms/step


In [216]:
rmse_lstm = np.sqrt(mean_squared_error(test_y3, pred_lstm))
rmse_lstm

1.0109636661267758

In [217]:
mean_squared_error(test_y3, pred_lstm)

1.0220475342284912